In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
from scipy.constants import Stefan_Boltzmann
import xarray as xr

import matplotlib.pyplot as plt
import seaborn as sns

import synthia as syn
from synthia.util import to_stacked_array, to_unstacked_dataset

from copulae import NormalCopula
import numpy as np

from scipy.stats import multivariate_normal

## Input data

In [ ]:
THIS_DIR = Path.cwd()
ds_input = xr.open_dataset(THIS_DIR.parents[1] / 'data' / 'nwp_saf_profiles_in.nc')

input_relevant = [
    'temperature_fl',
    'temperature_hl',
    'pressure_hl',
]

ds_true_in = ds_input[input_relevant]
ds_true_in

In [ ]:
# Create a sample dataset with n samples
N_SAMPLES = 1000
mvnorm = multivariate_normal(mean=[0, 0], cov=[[1, 0.5], 
                                                           [0.5, 1]])
# Generate random samples from multivariate normal with correlation .5
data = xr.DataArray(mvnorm.rvs(N_SAMPLES))

h = sns.jointplot(data[:, 0], data[:, 1], kind='kde', stat_func=None)
h.set_axis_labels('X1', 'X2', fontsize=16);

In [ ]:
generator = syn.CopulaDataGenerator()

parameterizer = syn.QuantileParameterizer(n_quantiles=100)

generator.fit(data, copula=syn.GaussianCopula(), parameterize_by=parameterizer)

samples = generator.generate(n_samples=100, uniformization_ratio=0, stretch_factor=2)

h = sns.jointplot(samples[:, 0], samples[:, 1], kind='kde', stat_func=None)
h.set_axis_labels('X1', 'X2', fontsize=16);

In [ ]:
data = xr.DataArray(mvnorm.rvs(N_SAMPLES))

rank = data.compute().rank(data.dims[0])
rank_standardized = rank / (rank.max(data.dims[0]) + 1)

cop = NormalCopula(2)
cop.fit(rank_standardized)
samples =  cop.random(N_SAMPLES)  # simulate random number


for i in range(2):
    samples[:,i] = np.quantile(data[:,i], q=samples[:, i], interpolation='linear')

h = sns.jointplot(samples[:, 0], samples[:, 1], kind='kde', stat_func=None)
h.set_axis_labels('X1', 'X2', fontsize=16);